In [1]:
import pandas as pd
import os
import datetime
import numpy as np

kanja = pd.read_csv("../data/kanja.csv")
syobyo = pd.read_csv("../data/syobyo.csv")
kenshin = pd.read_csv("../data/kenshin.csv")

# syobyo
syobyo_no_suspicion = syobyo.loc[syobyo.疑いフラグ!=1,['加入者id','標準病名','診療開始年月日']] 

# DME onset date
code_list = ['２型糖尿病性黄斑浮腫','糖尿病黄斑浮腫','のう胞様黄斑浮腫','糖尿病黄斑症']
df_inj = (syobyo_no_suspicion[syobyo_no_suspicion.標準病名.isin(code_list)].groupby('加入者id').min()
          [['診療開始年月日']].reset_index()
          .rename(columns={'診療開始年月日':'発症年月日'})
          .astype({'発症年月日':int})
         )
kanja = pd.merge(kanja, df_inj, on='加入者id', how='left')


# DM onset date
code_type2DM_list = ['２型糖尿病','インスリン抵抗性糖尿病','安定型糖尿病','若年２型糖尿病','２型糖尿病・昏睡合併あり','２型糖尿病性昏睡','２型糖尿病性低血糖性昏睡','２型糖尿病・ケトアシドーシス合併あり',
                     '２型糖尿病性アシドーシス','２型糖尿病性アセトン血症','２型糖尿病性ケトアシドーシス','２型糖尿病性ケトーシス','２型糖尿病・腎合併症あり','２型糖尿病性腎硬化症', '２型糖尿病性腎症',
                     '２型糖尿病性腎症第１期','２型糖尿病性腎症第２期','２型糖尿病性腎症第３期','２型糖尿病性腎症第３期Ａ','２型糖尿病性腎症第３期Ｂ','２型糖尿病性腎症第４期','２型糖尿病性腎症第５期',
                     '２型糖尿病性腎不全','２型糖尿病・眼合併症あり','２型糖尿病黄斑症','２型糖尿病黄斑浮腫','２型糖尿病性眼筋麻痺','２型糖尿病・眼合併症あり','２型糖尿病黄斑症','２型糖尿病性白内障',
                     '２型糖尿病性網膜症','増殖性糖尿病性網膜症・２型糖尿病','２型糖尿病・神経合併症あり','２型糖尿病性ニューロパチー','２型糖尿病性筋萎縮症',
                     '２型糖尿病性自律神経ニューロパチー','２型糖尿病性神経因性膀胱','２型糖尿病性神経痛',
                     '２型糖尿病性多発ニューロパチー','２型糖尿病性単ニューロパチー','２型糖尿病性末梢神経障害','２型糖尿病・末梢循環合併症あり','２型糖尿病性壊疽',
                     '２型糖尿病性血管障害','２型糖尿病性潰瘍','２型糖尿病性動脈硬化症','２型糖尿病性動脈閉塞症','２型糖尿病性末梢血管症','２型糖尿病性末梢血管障害',
                     '２型糖尿病・関節合併症あり','２型糖尿病・神経合併症あり', '2型糖尿病・腎合併症あり', '２型糖尿病・多発糖尿病性合併症あり','２型糖尿病・糖尿病性合併症あり',
                     '２型糖尿病・糖尿病性合併症なし','２型糖尿病・末梢循環合併症あり','２型糖尿病合併妊娠',' ２型糖尿病胃腸症','２型糖尿病虹彩炎','２型糖尿病性高コレステロール血症']

df_inj = (syobyo_no_suspicion[syobyo_no_suspicion.標準病名.isin(code_type2DM_list)].groupby('加入者id').min()
          [['診療開始年月日']].reset_index()
          .rename(columns={'診療開始年月日':'糖尿病発症年月日'})
          .astype({'糖尿病発症年月日':int})
         )

kanja = pd.merge(kanja, df_inj, on='加入者id', how='left')


# months from the start of observation to the DME onset date
kanja['観察開始から発症までの期間_月'] = np.nan
kanja.loc[kanja.is_mac==1,'観察開始から発症までの期間_月'] = (
           pd.to_datetime(kanja['発症年月日'], format='%Y%m%d').dt.to_period('M').astype('int')
         - pd.to_datetime(kanja['観察開始年月'], format='%Y%m').dt.to_period('M').astype('int')
           )

# months from the start of observation to the DM onset date
kanja['観察開始から糖尿病発症までの期間_月'] = (
           pd.to_datetime(kanja['糖尿病発症年月日'], format='%Y%m%d').dt.to_period('M').astype('int')
         - pd.to_datetime(kanja['観察開始年月'], format='%Y%m').dt.to_period('M').astype('int')
           )

# years from the start of observation to the DME onset date
kanja['観察開始から糖尿病発症までの期間_年']  = kanja['観察開始から糖尿病発症までの期間_月'] // 12

# age at the start of observation
kanja['観察開始時年齢'] = (
             pd.to_datetime(kanja['観察開始年月'],format='%Y%m').dt.to_period('M').astype('int')
           - pd.to_datetime(kanja['加入者生年月'],format='%Y%m').dt.to_period('M').astype('int') ) // 12   # 12ヶ月で割った商

# months of observation period
kanja['観察期間_月']  = (
             pd.to_datetime(kanja['観察終了年月'],format='%Y%m').dt.to_period('M').astype('int')
           - pd.to_datetime(kanja['観察開始年月'],format='%Y%m').dt.to_period('M').astype('int'))

# years of observation period
kanja['観察期間_年']  = (kanja['観察期間_月'] // 12)



# kenshin
# months from the start of observation to the first health check-up date
kenshin_first_date = (kenshin.groupby("加入者id").min()
        [['健診実施年月日']].reset_index()
          .rename(columns={'健診実施年月日':'初回健診日'})
          .astype({'初回健診日':int})
         )
kanja = pd.merge(kanja, kenshin_first_date, on='加入者id', how='left')
kanja['months_until_first_kenshin'] = np.nan
kanja.loc[~kanja.初回健診日.isna(),'months_until_first_kenshin']  = (
             pd.to_datetime(kanja['初回健診日'],format='%Y%m%d').dt.to_period('M').astype('int')
           - pd.to_datetime(kanja['観察開始年月'],format='%Y%m').dt.to_period('M').astype('int'))


# kenshin_times
kenshin_times = (kenshin.groupby("加入者id").count()
        [['健診実施年月日']].reset_index()
          .rename(columns={'健診実施年月日':'kenshin_times'})
         )
kanja = pd.merge(kanja, kenshin_times, on='加入者id', how='left')


# kenshin_times_before_onset (only DME)
df = pd.merge(kenshin,kanja.loc[kanja.is_mac==1,['加入者id','発症年月日']],on='加入者id')
df['健診実施年月日'] = pd.to_datetime(df['健診実施年月日'], format='%Y%m%d')
df['発症年月日'] = pd.to_datetime(df['発症年月日'], format='%Y%m%d')
df_bedore_onset = df.loc[df.健診実施年月日 < df.発症年月日]
df_kenshin_times = (df_bedore_onset[['加入者id','健診実施年月日']]
            .groupby('加入者id')
            .count()
            .reset_index()
            .rename(columns={'健診実施年月日':'kenshin_times_before_onset'}))
kanja = pd.merge(kanja, df_kenshin_times, on='加入者id', how='left')


kanja['durling_DM'] = kanja['観察期間_月']-kanja['観察開始から糖尿病発症までの期間_月']

kanja['発症年月日'] = pd.to_datetime(kanja['発症年月日'], format='%Y%m%d')
kanja['糖尿病発症年月日'] = pd.to_datetime(kanja['糖尿病発症年月日'], format='%Y%m%d')
kanja['初回健診日'] = pd.to_datetime(kanja['初回健診日'], format='%Y%m%d')

kanja.to_csv('../data/kanja_summary.csv',index=False)
